# The Cancer Gene Atlas Cancer (TCGA) - Pheochromocytoma and Paraganglioma (PCPG)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use


In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.5 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.3.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.2
pandas 0.22.0
clustergrammer_widget 1.9.0
seaborn 0.8.1
Wed May 09 10:40:36 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Pheochromocytoma and Paraganglioma [PCPG]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-QR-A705,4768,PCPG,TCGA,EBEF1483-E3FD-497C-B694-8C3A4E96102F,QR,63.0,NaN,366,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,"Yes, History of Synchronous/Bilateral Malignancy","Other, Other","Other, specify, Other, specify","Synchronous Malignancy, Synchronous Malignancy",NaN,NaN,NaN
TCGA-WB-A81E,4807,PCPG,TCGA,BA65B478-65D0-4C9A-A2C2-D32B29A1A9C4,WB,47.0,NaN,366,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,"Yes, History of Synchronous/Bilateral Malignancy",Gastrointestinal tract,"Other, specify",Prior Malignancy,NaN,NaN,NaN
TCGA-P8-A6RX,5126,PCPG,TCGA,78BB46F4-060D-45F3-B877-754A9D695798,P8,65.0,NaN,366,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-QR-A70P,5135,PCPG,TCGA,A39AC229-60D2-42C2-9721-7E0894C0AE38,QR,45.0,NaN,366,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,"Yes, History of Prior Malignancy",Head/Neck,"Other, specify",Prior Malignancy,NaN,NaN,NaN
TCGA-QR-A702,5136,PCPG,TCGA,C7522BED-B672-4F54-BBB7-31698F73F14D,QR,75.0,NaN,366,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(179, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_PCPG_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,482b10f0-9878-4ff2-a8cc-b43730571e6a,a6a5d412-eaa2-4866-9708-11b634cabf3f,2a8cadc8-be66-4496-af36-6dcf895e823a,9f62b7e1-fb36-43fb-8cc1-b0788e7961fc,12f83f49-475c-4b62-a2d3-65958a508a8b,fd72ba31-4e90-4490-9f88-e9991de006ad,4f5d07d6-68a7-4cdf-b178-c0c9d0876256,10f1c6db-0edd-4db9-ad14-c12993f16e9f,a2677171-dca5-4b95-9e19-3cb6e13c6db2,f6a583bc-49ee-4bd2-9fb5-b2ecf118094c,...,2ac8a7ed-ff86-4de7-8dc5-cf19850449be,687253da-d65a-49b4-b3eb-c46b6b819c17,4a297ccf-a1c3-41f1-8cbe-be56698a1ba8,ee5c43c5-24c0-4979-bf54-afe56b975be0,b912e811-da52-41be-a2e0-61186bdb3ec7,b49ba954-e23b-4be8-b5d6-e02fdb49cb24,13ad780f-74f1-4974-b70e-f53f9089d182,883f6cb5-5843-43ae-8696-ae8dcf1377ef,7df6df19-a256-42a8-a937-7cff36b03311,d8a510e5-d7fd-4ced-a5c6-486b07e4e120
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,157.261000,168.58200,111.80900,88.48420,173.17700,107.27400,126.7460,99.71290,152.227000,170.89800,...,65.406200,102.46400,140.906000,99.36580,7.873370e+01,111.50800,114.92300,71.84180,108.5450,93.93180
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,0.000000,7.73397,30.65040,18.52740,26.79470,36.85460,28.0703,52.29930,17.155100,36.34830,...,15.090000,15.04970,17.392300,30.09740,6.737660e+00,36.74260,45.71440,15.68610,40.3586,9.29763
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.971900,5.69579,20.21630,18.28070,13.25480,12.24880,30.6747,13.66060,14.491600,16.22950,...,3.926010,14.88720,8.990880,8.01032,1.103300e+01,17.91460,9.81285,17.24520,11.5628,7.70253
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,28.195400,35.84520,36.13050,45.37790,38.64740,41.98160,42.5818,33.34570,26.013600,36.37300,...,21.910400,41.82110,45.797800,25.57640,1.950140e+01,35.25630,50.44520,28.63190,42.1358,18.61670
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.024145,0.35254,1.45739,5.77534,1.25056,3.69749,11.4744,4.07096,0.610323,1.40435,...,0.129773,1.05556,0.254762,1.45002,1.464570e-07,1.42098,28.13800,1.23503,10.4331,1.51953


In [19]:
matrix.shape

(199169, 187)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,482b10f0-9878-4ff2-a8cc-b43730571e6a,a6a5d412-eaa2-4866-9708-11b634cabf3f,2a8cadc8-be66-4496-af36-6dcf895e823a,9f62b7e1-fb36-43fb-8cc1-b0788e7961fc,12f83f49-475c-4b62-a2d3-65958a508a8b,fd72ba31-4e90-4490-9f88-e9991de006ad,4f5d07d6-68a7-4cdf-b178-c0c9d0876256,10f1c6db-0edd-4db9-ad14-c12993f16e9f,a2677171-dca5-4b95-9e19-3cb6e13c6db2,f6a583bc-49ee-4bd2-9fb5-b2ecf118094c,...,2ac8a7ed-ff86-4de7-8dc5-cf19850449be,687253da-d65a-49b4-b3eb-c46b6b819c17,4a297ccf-a1c3-41f1-8cbe-be56698a1ba8,ee5c43c5-24c0-4979-bf54-afe56b975be0,b912e811-da52-41be-a2e0-61186bdb3ec7,b49ba954-e23b-4be8-b5d6-e02fdb49cb24,13ad780f-74f1-4974-b70e-f53f9089d182,883f6cb5-5843-43ae-8696-ae8dcf1377ef,7df6df19-a256-42a8-a937-7cff36b03311,d8a510e5-d7fd-4ced-a5c6-486b07e4e120
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,157.261000,168.58200,111.80900,88.48420,173.17700,107.27400,126.7460,99.71290,152.227000,170.89800,...,65.406200,102.46400,140.906000,99.36580,7.873370e+01,111.50800,114.92300,71.84180,108.5450,93.93180
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,0.000000,7.73397,30.65040,18.52740,26.79470,36.85460,28.0703,52.29930,17.155100,36.34830,...,15.090000,15.04970,17.392300,30.09740,6.737660e+00,36.74260,45.71440,15.68610,40.3586,9.29763
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.971900,5.69579,20.21630,18.28070,13.25480,12.24880,30.6747,13.66060,14.491600,16.22950,...,3.926010,14.88720,8.990880,8.01032,1.103300e+01,17.91460,9.81285,17.24520,11.5628,7.70253
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,28.195400,35.84520,36.13050,45.37790,38.64740,41.98160,42.5818,33.34570,26.013600,36.37300,...,21.910400,41.82110,45.797800,25.57640,1.950140e+01,35.25630,50.44520,28.63190,42.1358,18.61670
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.024145,0.35254,1.45739,5.77534,1.25056,3.69749,11.4744,4.07096,0.610323,1.40435,...,0.129773,1.05556,0.254762,1.45002,1.464570e-07,1.42098,28.13800,1.23503,10.4331,1.51953


In [22]:
matrix.shape

(79930, 187)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-W2-A7H7-01,TCGA-WB-A81V-01,TCGA-SR-A6MZ-01,TCGA-SR-A6MS-01,TCGA-S7-A7X2-01,TCGA-S7-A7WR-01,TCGA-WB-A822-01,TCGA-WB-A81S-01,TCGA-SP-A6QJ-01,TCGA-WB-A81Q-01,...,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,TCGA-QR-A70E-01,TCGA-QR-A6GU-01,TCGA-TT-A6YO-01,TCGA-W2-A7H5-01,TCGA-S7-A7X0-01,TCGA-P8-A5KD-11,TCGA-P7-A5NY-01,TCGA-QR-A706-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,157.261000,168.58200,111.80900,88.48420,173.17700,107.27400,126.7460,99.71290,152.227000,170.89800,...,65.406200,102.46400,140.906000,99.36580,7.873370e+01,111.50800,114.92300,71.84180,108.5450,93.93180
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,0.000000,7.73397,30.65040,18.52740,26.79470,36.85460,28.0703,52.29930,17.155100,36.34830,...,15.090000,15.04970,17.392300,30.09740,6.737660e+00,36.74260,45.71440,15.68610,40.3586,9.29763
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.971900,5.69579,20.21630,18.28070,13.25480,12.24880,30.6747,13.66060,14.491600,16.22950,...,3.926010,14.88720,8.990880,8.01032,1.103300e+01,17.91460,9.81285,17.24520,11.5628,7.70253
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,28.195400,35.84520,36.13050,45.37790,38.64740,41.98160,42.5818,33.34570,26.013600,36.37300,...,21.910400,41.82110,45.797800,25.57640,1.950140e+01,35.25630,50.44520,28.63190,42.1358,18.61670
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.024145,0.35254,1.45739,5.77534,1.25056,3.69749,11.4744,4.07096,0.610323,1.40435,...,0.129773,1.05556,0.254762,1.45002,1.464570e-07,1.42098,28.13800,1.23503,10.4331,1.51953


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-W2-A7H7-01,TCGA-WB-A81V-01,TCGA-SR-A6MZ-01,TCGA-SR-A6MS-01,TCGA-S7-A7X2-01,TCGA-S7-A7WR-01,TCGA-WB-A822-01,TCGA-WB-A81S-01,TCGA-SP-A6QJ-01,TCGA-WB-A81Q-01,...,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,TCGA-QR-A70E-01,TCGA-QR-A6GU-01,TCGA-TT-A6YO-01,TCGA-W2-A7H5-01,TCGA-S7-A7X0-01,TCGA-P8-A5KD-11,TCGA-P7-A5NY-01,TCGA-QR-A706-01
ARF5,157.261000,168.58200,111.80900,88.48420,173.17700,107.27400,126.7460,99.71290,152.227000,170.89800,...,65.406200,102.46400,140.906000,99.36580,7.873370e+01,111.50800,114.92300,71.84180,108.5450,93.93180
M6PR,0.000000,7.73397,30.65040,18.52740,26.79470,36.85460,28.0703,52.29930,17.155100,36.34830,...,15.090000,15.04970,17.392300,30.09740,6.737660e+00,36.74260,45.71440,15.68610,40.3586,9.29763
ESRRA,15.971900,5.69579,20.21630,18.28070,13.25480,12.24880,30.6747,13.66060,14.491600,16.22950,...,3.926010,14.88720,8.990880,8.01032,1.103300e+01,17.91460,9.81285,17.24520,11.5628,7.70253
FKBP4,28.195400,35.84520,36.13050,45.37790,38.64740,41.98160,42.5818,33.34570,26.013600,36.37300,...,21.910400,41.82110,45.797800,25.57640,1.950140e+01,35.25630,50.44520,28.63190,42.1358,18.61670
CYP26B1,0.024145,0.35254,1.45739,5.77534,1.25056,3.69749,11.4744,4.07096,0.610323,1.40435,...,0.129773,1.05556,0.254762,1.45002,1.464570e-07,1.42098,28.13800,1.23503,10.4331,1.51953


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_pcpg_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-W2-A7H7-01,TCGA-WB-A81V-01,TCGA-SR-A6MZ-01,TCGA-SR-A6MS-01,TCGA-S7-A7X2-01,TCGA-S7-A7WR-01,TCGA-WB-A822-01,TCGA-WB-A81S-01,TCGA-SP-A6QJ-01,TCGA-WB-A81Q-01,...,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,TCGA-QR-A70E-01,TCGA-QR-A6GU-01,TCGA-TT-A6YO-01,TCGA-W2-A7H5-01,TCGA-S7-A7X0-01,TCGA-P8-A5KD-11,TCGA-P7-A5NY-01,TCGA-QR-A706-01
A1BG,0.047756,0.047756,0.047756,0.036432,0.047756,0.180451,0.047756,0.047756,0.047756,0.034163,...,0.047756,0.047756,0.047756,0.047756,0.047756,0.025120,0.047756,0.047756,0.047756,0.047756
A1CF,0.021305,0.011185,0.024703,0.015081,0.027481,0.042058,0.019438,0.035262,0.014271,0.009377,...,0.005307,0.012581,0.012798,0.006080,0.002326,0.064170,0.020311,0.002419,0.007448,0.029374
A2M,100.445000,46.900900,142.261000,90.664800,133.054000,118.025000,357.781000,111.266000,72.780900,64.201200,...,64.936300,87.534800,55.560700,190.572000,19.448000,57.386300,304.740000,63.608800,82.488600,215.298000
A2ML1,0.027354,0.069574,0.085435,0.016037,0.045288,0.273566,0.078836,0.081334,0.055963,0.227166,...,0.038881,0.024423,0.118162,0.038366,0.034717,0.152849,0.109939,0.070154,0.237578,0.149097
A3GALT2,0.049850,0.049850,0.023639,0.049850,0.049850,0.026310,0.068023,0.047164,0.049850,0.049850,...,0.049850,0.049850,0.062147,0.047713,0.019026,0.147788,0.032048,0.049850,0.049850,0.043370


In [34]:
normalized_matrix.shape

(18882, 187)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-W2-A7H7-01,TCGA-WB-A81V-01,TCGA-SR-A6MZ-01,TCGA-SR-A6MS-01,TCGA-S7-A7X2-01,TCGA-S7-A7WR-01,TCGA-WB-A822-01,TCGA-WB-A81S-01,TCGA-SP-A6QJ-01,TCGA-WB-A81Q-01,...,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,TCGA-QR-A70E-01,TCGA-QR-A6GU-01,TCGA-TT-A6YO-01,TCGA-W2-A7H5-01,TCGA-S7-A7X0-01,TCGA-P8-A5KD-11,TCGA-P7-A5NY-01,TCGA-QR-A706-01
A1BG,-1.041968,-1.111005,-1.134451,-1.219560,-1.137436,-0.565129,-1.147542,-1.157023,-1.155203,-1.278194,...,-1.194600,-1.135081,-1.152909,-1.168047,-1.093832,-1.392571,-1.150826,-1.174024,-1.138041,-1.147891
A1CF,-1.392519,-1.741388,-1.420734,-1.602596,-1.377424,-1.197640,-1.537922,-1.288745,-1.679776,-1.839700,...,-2.148786,-1.714399,-1.724794,-2.063197,-2.406326,-0.985265,-1.522130,-2.469485,-1.945054,-1.358952
A2M,2.280934,1.881149,2.339608,2.176398,2.307565,2.250486,2.727048,2.210313,2.027787,1.995797,...,1.938860,2.128073,1.912831,2.432985,1.516016,1.966210,2.654077,1.950466,2.099326,2.506122
A2ML1,-1.283969,-0.947584,-0.881840,-1.575909,-1.160478,-0.384426,-0.929846,-0.925776,-1.086329,-0.455402,...,-1.283889,-1.426316,-0.759458,-1.263131,-1.232323,-0.608335,-0.788701,-1.006997,-0.441262,-0.653449
A3GALT2,-1.023331,-1.092368,-1.439857,-1.083377,-1.118799,-1.401361,-0.993910,-1.162437,-1.136567,-1.114082,...,-1.175964,-1.116444,-1.038518,-1.168436,-1.493512,-0.622959,-1.324056,-1.155388,-1.119405,-1.189727


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-W2-A7H7-01,TCGA-WB-A81V-01,TCGA-SR-A6MZ-01,TCGA-SR-A6MS-01,TCGA-S7-A7X2-01,TCGA-S7-A7WR-01,TCGA-WB-A822-01,TCGA-WB-A81S-01,TCGA-SP-A6QJ-01,TCGA-WB-A81Q-01,...,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,TCGA-QR-A70E-01,TCGA-QR-A6GU-01,TCGA-TT-A6YO-01,TCGA-W2-A7H5-01,TCGA-S7-A7X0-01,TCGA-P8-A5KD-11,TCGA-P7-A5NY-01,TCGA-QR-A706-01
A1BG,-1.276210,-1.094593,-1.200375,-1.214724,-1.153714,-0.622342,-1.149426,-1.194568,-1.155190,-1.328053,...,-1.036283,-1.183321,-1.135704,-1.108657,-0.947313,-1.406093,-1.190561,-1.047726,-1.096832,-1.163779
A1CF,-1.693612,-1.710505,-1.486462,-1.596009,-1.398393,-1.232334,-1.548580,-1.319894,-1.672288,-1.865319,...,-2.032623,-1.826570,-1.665066,-2.029558,-2.290978,-1.036554,-1.585787,-2.361576,-1.871925,-1.367849
A2M,2.285917,1.871187,2.303855,2.101143,2.206951,2.169210,2.692731,2.249777,2.098937,1.893376,...,2.264607,2.068500,1.979365,2.509145,1.642291,1.928708,2.607189,2.054037,1.961603,2.580105
A2ML1,-1.559144,-0.917712,-0.963906,-1.564855,-1.178271,-0.470022,-0.936076,-0.982715,-1.083555,-0.551018,...,-1.136294,-1.488209,-0.769983,-1.204316,-1.120593,-0.672419,-0.838319,-0.849040,-0.440584,-0.682591
A3GALT2,-1.256834,-1.079163,-1.508399,-1.069572,-1.136569,-1.433170,-1.003356,-1.198818,-1.139443,-1.168455,...,-1.017887,-1.165895,-1.029715,-1.109694,-1.398726,-0.687684,-1.368520,-1.031907,-1.074396,-1.200987


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-W2-A7H7-01,TCGA-WB-A81V-01,TCGA-SR-A6MZ-01,TCGA-SR-A6MS-01,TCGA-S7-A7X2-01,TCGA-S7-A7WR-01,TCGA-WB-A822-01,TCGA-WB-A81S-01,TCGA-SP-A6QJ-01,TCGA-WB-A81Q-01,...,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,TCGA-QR-A70E-01,TCGA-QR-A6GU-01,TCGA-TT-A6YO-01,TCGA-W2-A7H5-01,TCGA-S7-A7X0-01,TCGA-P8-A5KD-11,TCGA-P7-A5NY-01,TCGA-QR-A706-01
A1BG,-1.368525,0.680357,-0.513009,-0.674880,0.013391,6.007978,0.061765,-0.447493,-0.003251,-1.953385,...,1.338168,-0.320614,0.216577,0.521692,2.341874,-2.833773,-0.402286,1.209076,0.655100,-0.100150
A1CF,-0.160358,-0.225087,0.633390,0.213635,0.970850,1.607151,0.395371,1.271642,-0.078647,-0.818296,...,-1.459366,-0.669820,-0.050977,-1.447623,-2.449321,2.357333,0.252804,-2.719836,-0.843609,1.087887
A2M,0.411478,-1.150486,0.479035,-0.284423,0.114076,-0.028065,1.943626,0.275368,-0.292730,-1.066917,...,0.331219,-0.407361,-0.743065,1.252204,-2.012559,-0.933849,1.621457,-0.461832,-0.809959,1.519453
A2ML1,-2.427069,-0.028374,-0.201122,-2.448426,-1.002758,1.645806,-0.097047,-0.271458,-0.648559,1.342915,...,-0.845781,-2.161801,0.524072,-1.100157,-0.787066,0.888926,0.268523,0.228432,1.755893,0.850883
A3GALT2,-0.719388,0.563287,-2.535533,0.632525,0.148849,-1.992430,1.110565,-0.300548,0.128100,-0.081349,...,1.005657,-0.062865,0.920268,0.342867,-1.743766,3.389517,-1.525694,0.904441,0.597702,-0.316209


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(18882, 187)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

(18459, 187)

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18370, 187)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Filtered Matrix

In [52]:
filename = path+'tcga_pcpg_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18370, 2)

# Save Gene List

In [56]:
filename = path+'tcga_pcpg_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [90]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [91]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [92]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-P7-A5NX-01,78,FEMALE,NaN,NaN,NaN,NaN,Pheochromocytoma,Primary Solid Tumor,Adrenal gland,NaN
TCGA-P7-A5NY-01,21,FEMALE,NaN,NaN,NaN,NaN,Pheochromocytoma,Primary Solid Tumor,Adrenal gland,NaN
TCGA-P7-A5NY-05,21,FEMALE,NaN,NaN,NaN,NaN,Pheochromocytoma,Additional - New Primary,Adrenal gland,NaN
TCGA-P8-A5KC-01,48,MALE,NaN,NaN,NaN,NaN,Pheochromocytoma,Primary Solid Tumor,Adrenal gland,NaN
TCGA-P8-A5KC-11,48,MALE,NaN,NaN,NaN,NaN,Pheochromocytoma,Solid Tissue Normal,Adrenal gland,NaN


In [93]:
attribute_list.shape

(187, 10)

# Save Attribute List

In [94]:
filename = path+'tcga_pcpg_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [63]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [64]:
standard_matrix.head()

,TCGA-P7-A5NX-01,TCGA-P7-A5NY-01,TCGA-P7-A5NY-05,TCGA-P8-A5KC-01,TCGA-P8-A5KC-11,TCGA-P8-A5KD-01,TCGA-P8-A5KD-11,TCGA-P8-A6RX-01,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,...,TCGA-WB-A81Q-01,TCGA-WB-A81R-01,TCGA-WB-A81S-01,TCGA-WB-A81T-01,TCGA-WB-A81V-01,TCGA-WB-A81W-01,TCGA-WB-A820-01,TCGA-WB-A821-01,TCGA-WB-A822-01,TCGA-XG-A823-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.508176,0.497044,0.026286,-0.636475,0.293695,-0.924988,0.700386,-0.593711,0.732480,-0.326508,...,-0.796762,0.101201,-0.412090,-0.978363,0.550477,-0.230276,-0.615115,0.283026,0.036967,0.711082
A1CF,0.732846,-0.561311,0.497408,-0.411715,0.486673,0.240619,-0.935304,-0.454437,-0.721605,-0.433059,...,-0.518488,0.176361,0.775749,0.957867,-0.251313,0.840004,-0.165795,-0.379619,0.315538,-0.924622
A2M,0.145527,-0.560158,0.412964,0.905492,-0.111175,0.273826,-0.335696,0.894760,0.263163,-0.292920,...,-0.677689,-0.773858,0.209665,0.059912,-0.688347,-0.720440,-0.014978,0.305964,0.916175,0.605577
A2ML1,-0.314043,0.852134,-0.730873,0.381106,-0.933891,0.113679,0.209913,0.680734,-0.602643,-0.901841,...,0.744992,0.712858,-0.303330,-0.410300,-0.036115,0.370385,0.541558,-0.891164,-0.089502,-0.773609
A3GALT2,-0.347116,0.433625,0.080604,-0.518047,-0.496663,-0.315002,0.626221,-0.389845,0.690474,-0.122537,...,-0.143936,-0.742537,-0.304279,0.112712,0.390856,-0.005012,-0.432618,0.219646,0.711901,0.647641


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [65]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [66]:
filename = path+'tcga_pcpg_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [67]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [68]:
tertiary_matrix.head()

,TCGA-P7-A5NX-01,TCGA-P7-A5NY-01,TCGA-P7-A5NY-05,TCGA-P8-A5KC-01,TCGA-P8-A5KC-11,TCGA-P8-A5KD-01,TCGA-P8-A5KD-11,TCGA-P8-A6RX-01,TCGA-P8-A6RY-01,TCGA-PR-A5PF-01,...,TCGA-WB-A81Q-01,TCGA-WB-A81R-01,TCGA-WB-A81S-01,TCGA-WB-A81T-01,TCGA-WB-A81V-01,TCGA-WB-A81W-01,TCGA-WB-A820-01,TCGA-WB-A821-01,TCGA-WB-A822-01,TCGA-XG-A823-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [69]:
filename = path+'tcga_pcpg_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [70]:
name = 'tcga_pcpg_gene_up_set'

In [71]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [72]:
name = 'tcga_pcpg_gene_down_set'

In [73]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [74]:
name = 'tcga_pcpg_attribute_up_set'

In [75]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [76]:
name = 'tcga_pcpg_attribute_down_set'

In [77]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [79]:
attribute_normalized_matrix = normalized_matrix.copy()

In [80]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [81]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [82]:
attribute_similarity_matix.head()

,"(Donor:TCGA-P7-A5NX, Age:78.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P7-A5NY, Age:21.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P7-A5NY, Age:21.0, Gender:FEMALE, SampleType:Additional - New Primary, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P8-A5KC, Age:48.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P8-A5KC, Age:48.0, Gender:MALE, SampleType:Solid Tissue Normal, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P8-A5KD, Age:35.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P8-A5KD, Age:35.0, Gender:FEMALE, SampleType:Solid Tissue Normal, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-P8-A6RX, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Paraganglioma, Tissue:Extra-adrenal Site, Tumor:nan, Gradenan)","(Donor:TCGA-P8-A6RY, Age:55.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-PR-A5PF, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)",...,"(Donor:TCGA-WB-A81Q, Age:44.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A81R, Age:55.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A81S, Age:59.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A81T, Age:45.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A81V, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A81W, Age:41.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A820, Age:40.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A821, Age:73.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-WB-A822, Age:23.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)","(Donor:TCGA-XG-A823, Age:25.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-P7-A5NX, Age:78.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)",1.000000,0.049705,0.056774,0.107914,-0.051136,0.114092,-0.055233,0.103004,0.078326,0.115266,...,0.065677,0.037986,0.066063,0.102787,0.054861,0.232689,0.115078,0.039784,0.018014,-0.096693
"(Donor:TCGA-P7-A5NY, Age:21.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Pheochromocytoma, Tissue:Adrenal gland, Tumor:nan, Gradenan)",0.049705,1.000000,0.164970,0.057929,-0.026098,0.154202,-0.051782,-0.072656,0.033471,0.101914,...,0.091260,0.157917,0.095776,0.248165,0.083552,0.207732,0.153507,0.015302,-0.057283,-0.040059
"(Donor:TCGA-P7-A5NY, Age:21.0, Gender:FEMALE,

# Save Attribute Similarity Matrix

In [83]:
filename = path+'tcga_pcpg_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [84]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

clustergrammer_widget(network='{"col_nodes": [{"cat_7_index": 0, "clust": 120, "cat-2": "SampleType:Primary So…

# Create Gene Similarity Matrix

In [85]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [86]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.009353,-0.105048,0.036159,0.078705,0.040771,0.072077,0.008803,-0.169394,0.028646,...,-0.025596,0.020484,-0.020132,-0.026515,-0.030981,-0.024622,-0.013350,0.003642,-0.013603,-0.052566
A1CF,-0.009353,1.000000,-0.020614,0.119319,-0.031384,-0.213288,0.085908,-0.044049,0.183219,0.054424,...,0.183129,-0.008878,0.172093,0.208747,-0.100174,0.131905,-0.059343,-0.193794,0.168226,0.129942
A2M,-0.105048,-0.020614,1.000000,0.016939,-0.073950,0.581017,-0.084105,-0.159541,0.200131,-0.098910,...,-0.154004,-0.271786,-0.118565,-0.278431,0.126952,-0.125534,0.235883,0.646484,-0.027294,0.194634
A2ML1,0.036159,0.119319,0.016939,1.000000,0.003041,-0.132320,0.017038,-0.058433,-0.042721,0.023788,...,-0.072696,-0.130216,0.047498,0.092492,0.076964,0.288793,-0.045735,-0.085875,0.040380,0.055981
A3GALT2,0.078705,-0.031384,-0.073950,0.003041,1.000000,0.081466,0.049274,-0.067234,-0.299159,-0.003229,...,0.022148,0.162673,-0.116763,-0.053629,-0.025750,0.057188,-0.030409,0.141579,-0.073598,-0.152246


# Save Gene Similarity Matrix

In [87]:
filename = path+'tcga_pcpg_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [88]:
name = 'tcga_pcpg_gene_attribute_edge_list'

In [89]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  187 Out of 187   

 The number of statisticaly relevent gene-attribute associations is: 172133
